### Generating lyrics
For trainiing a GPT-2 model we will use gpt-2-simple, which is makes it easy to retrain a model with Wu-Tang lyrics. Go check https://github.com/minimaxir/gpt-2-simple out!

In [ ]:
import gpt_2_simple as gpt2
import os
import requests

model_name = "124M"
file_name = "wu_tang_songs.txt"

Download the pre-trained model, copied from https://github.com/openai/gpt-2/blob/master/download_model.py

In [ ]:
from tqdm import tqdm

model = model_name

subdir = os.path.join('models', model)
if not os.path.exists(subdir):
    os.makedirs(subdir)
subdir = subdir.replace('\\','/') # needed for Windows

for filename in ['checkpoint','encoder.json','hparams.json','model.ckpt.data-00000-of-00001', 'model.ckpt.index', 'model.ckpt.meta', 'vocab.bpe']:

    r = requests.get("https://openaipublic.blob.core.windows.net/gpt-2/" + subdir + "/" + filename, stream=True)

    with open(os.path.join(subdir, filename), 'wb') as f:
        file_size = int(r.headers["content-length"])
        chunk_size = 1000
        with tqdm(ncols=100, desc="Fetching " + filename, total=file_size, unit_scale=True) as pbar:
            # 1k for chunk_size, since Ethernet packet size is around 1500 bytes
            for chunk in r.iter_content(chunk_size=chunk_size):
                f.write(chunk)
                pbar.update(chunk_size)

Finetune model, to train several models change *run_name*

In [ ]:
sess = gpt2.start_tf_sess()
gpt2.finetune(sess,
              file_name,
              model_name=model_name,
              run_name='run2',
              steps=100)   # steps is max number of training steps

Generate samples. I've noticed that higher values of temperature generates better results.

In [ ]:
gpt2.generate(sess, temperature=0.9, run_name='run2')